In [3]:
import pandas as pd
import math
from scipy import stats
from scipy import spatial
import numpy as np
import matplotlib.pyplot as plt
from pointpats import PointPattern, as_window
from pointpats import PoissonPointProcess as csr
from pointpats.geometry import (prepare_hull as _prepare_hull, area as _area,
    build_best_tree as _build_best_tree,
    prepare_hull as _prepare_hull,
    TREE_TYPES,
)
from scipy.spatial import distance
from pointpats.ripley import _prepare # very important to prepare data :)

/Users/gocchini/opt/anaconda3/envs/paper_3/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<string>:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.ht

In [117]:
points = [[66.22, 32.54], [22.52, 22.39], [31.01, 81.21],
          [9.47, 31.02],  [30.78, 60.10], [75.21, 58.93],
          [79.26,  7.68], [8.23, 39.93],  [98.73, 77.17],
          [89.78, 42.53], [65.19, 92.08], [54.46, 8.48]]
#points is my arg! 
support = [0.        , 0.00584394, 0.01168788, 0.01753182, 0.02337576,
       0.0292197 , 0.03506364, 0.04090759, 0.04675153, 0.05259547]
pp = PointPattern(points)

/Users/gocchini/opt/anaconda3/envs/paper_3/lib/python3.10/site-packages/libpysal/cg/shapes.py:1492: FutureWarning: Objects based on the `Geometry` class will deprecated and removed in a future version of libpysal.
  warnings.warn(dep_msg, FutureWarning)
/Users/gocchini/opt/anaconda3/envs/paper_3/lib/python3.10/site-packages/libpysal/cg/shapes.py:1208: FutureWarning: Objects based on the `Geometry` class will deprecated and removed in a future version of libpysal.
  warnings.warn(dep_msg, FutureWarning)


In [18]:
pp.points['coords'] = points

In [53]:
pp.points

,x,y,coords
0,66.22,32.54,"[66.22, 32.54]"
1,22.52,22.39,"[22.52, 22.39]"
2,31.01,81.21,"[31.01, 81.21]"
3,9.47,31.02,"[9.47, 31.02]"
4,30.78,60.10,"[30.78, 60.1]"
5,75.21,58.93,"[75.21, 58.93]"
6,79.26,7.68,"[79.26, 7.68]"
7,8.23,39.93,"[8.23, 39.93]"
8,98.73,77.17,"[98.73, 77.17]"
9,89.78,42.53,"[89.78, 42.53]"


In [29]:
kernel = stats.gaussian_kde(np.vstack([pp.points['x'].array, pp.points['y'].array]))

In [34]:
lambda_values = kernel.pdf(kernel.dataset)

In [119]:
def find_lambda(kernel, x, y):
    values = np.vstack([x, y])
    return kernel.pdf(values)

def fun_find_kinhom(distance, support, denom):
    kinhom_values = []
    for x in support:
        if x > distance:
            x = 1/denom
        else: 
            x = 0 
        kinhom_values.append(x)
    return np.array(kinhom_values)

In [120]:
def kinhom(coordinates, support):
    all_pairs = [(a, b) for idx, a in enumerate(coordinates) for b in coordinates[idx + 1:]]
    df = pd.DataFrame(all_pairs)
    df.columns = [['x', 'y']]
    df['lambda_values_x'] = df.apply(lambda x: find_lambda(kernel, x['x'][0], x['x'][1]), axis = 1) 
    df['lambda_values_y'] = df.apply(lambda x: find_lambda(kernel, x['y'][0], x['y'][1]), axis = 1)
    df['denom'] = df.apply(lambda x: x['lambda_values_x']*x['lambda_values_y'], axis =1) 
    df['distance'] = df.apply(lambda x: distance.euclidean(x['x'], x['y']), axis = 1)
    df['support'] = df.apply(lambda row: support, axis = 1)
    df['kinhom'] = df.apply(lambda x: fun_find_kinhom(x['distance'], x['support'], x['denom']), axis = 1)
    n_pairs_less_than_d = df['kinhom'].values.tolist()
    internal_argument = np.sum(n_pairs_less_than_d, 0)
    return df, n_pairs_less_than_d, internal_argument

In [124]:
results = kinhom(points, support)